# 🔥 Desafio - **Avaliação de conformidade - Monitoramento hídrico**

## Enunciado:

- As tabelas **`parameter_sample`** e **`parameter_result`** armazenam dados de amostras e resultados, respectivamente, de monitoramento de qualidade hídrica.
- As tabelas **`zlegislation_record`** e **`zstandard_reference`** armazenam padrões legais e os limites estipulados para cada parâmetro, respectivamente.
- A tabela **`zparameters`** armazena os parâmetros de monitoramento hídrico de qualidade, juntamente com suas propriedades.
- As amostras são derivadas dos pontos de monitoramento (tabela `station`) e os resultados são derivados das amostras. 

O esquema a seguir ilustra os relacionamentos entre as tabelas (alguns campos foram omitidos para melhorar a visualização).

![station_samples_results](../../img/desafios/aula04/sql_diagram.png)

> Note que não foi estabelecido um relacionamento direto entre o conjunto de tabelas com dados do monitoramento (`station`, `parameter_sample` e `parameter_result`) e as tabelas com legislações e limites (`zlegislation_record` e `zstandard_reference`). A modelagem destes dados no projeto foi realizada de maneira simplificada, não considerando totalmente a normalização de alguns campos comuns, como o da matriz.

Como citado no desafio anterior, uma análise comum e de muita importância inerente aos dados de qualidade hídrica, é a **avaliação de conformidade legal**, através da comparação das medições a valores de referência.

## Objetivo
O desafio é escrever uma consulta SQL para alimentar um *dashboard* de análise de conformidade hídrica, em que um dos indicadores é a quantidade de resultados em desacordo com os limites legais de uma determinada legislação.
Deve ser possível **selecionar uma ou mais legislações** (assim como pontos e/ou parâmetros) pela interface da ferramenta analítica e o indicador deve retornar o total de desvios para as legislações selecionadas.

Há diferentes formas de resolver este problema, sendo um deles a criação de um modelo *star-schema* (fato-dimensão), a partir das tabelas envolvidas. Vamos adotar esta abordagem.

Os seguintes campos devem estar presentes no modelo de dados:
- Codigo HGA: Nome HGA do ponto de monitoramento (station.Name)
- Tipo Ponto: Tipo do ponto de monitoramento (station.stn_type)
- Local: Localização do ponto de monitoramento (station.zlocation)
- Matriz: Matriz do ponto de monitoramento (station.zmatrix)
- Enquadramento: Classe de enquadramento do ponto de monitoramento (station.zlegal_framework)
- ID Amostra: Identificador da amostra de qualidade (parameter_sample.sample_id ou parameter_result.sample_id)
- Data Amostra: Data de amostragem (parameter_sample.sample_date)
- Parametro: Nome do parametro químico analisado (parameter_result.chemical_name ou zparameters.chemical_name)
- Grupo Parametro: Agrupamento dos parâmetros químicos de acordo com sua natureza (zparameters.zparameter_group)
- Resultado: Valor numérico obtido na análise do parâmetro (parameter_result.result_value)
- Unidade: Unidade referente ao parâmetro analisado (parameter_result.result_unit)
- Legislação: Nome da legislação a qual os resultados estão sendo comparados (zlegislation_record.zreference_legislation ou zstandard_reference.zreference_legislation)
- Limite Min: Valor minimo estabelecido pela legislação, para o parâmetro em questão (zstandard_reference.zminimum_limit)
- Limite Max: Valor maximo estabelecido pela legislação, para o parâmetro em questão (zstandard_reference.zmaximum_limit)
- **Validacao: [Conforme/Não Conforme/Sem Comparação] Cálculo indicando se o resultado está de acordo ou não com os respectivos limites (??)**

Obs.: Os resultados de um ponto devem ser comparados aos limites **de todas as legislações pertinentes**, considerando os campos **matriz** e **enquadramento**. Ex.: Os pontos de **água superficiais** de enquadramento **classe 2** devem ser comparados às legislações `CONAMA 357/2005_Doce II` e `COPAM 01/2008_Doce II`.

****

### Entendendo o problema

Neste caso, precisamos combinar tabelas utilizando algumas regras lógicas que **não estão presentes na estrutura do próprio banco de dados**. Por exemplo, não há um relacionamento físico entre o campo `station.zmatrix` e o campo `zlegislation_record.zmatrix` ... porém, esta relação é fundamental para antender ao objetivo.

Ao utilizar ferramentas como o *Query Designer*, esta lógica não é considerada automaticamente.

![Query Designer](../../img/desafios/aula04/query_designer.png)

> Esta consulta não retornará os registros corretos..

In [ ]:
-- 💡 Dica 1: É esperado que a tabela final possua mais linhas que as tabelas envolvidas, uma vez que um resultado pode ter correspondência a limites de múltiplas legislações
-- 💡 Dica 2: Nem todos os parâmetros possuem limites definidos nas legislações.. Como garantir que eles não sejam excluídos da tabela final ?
    


# 🤯 Desafio desafiador

A consulta proposta anteriormente envolve a mesclagem (JOIN) de várias tabelas - `parameter_sample`, `parameter_result`, `station`, `zparameters`, `zlegislation_record` e `zstandard_reference` - o que pode torná-la computacionalmente cara.

Analise o plano de execução da consulta e avalie como ela pode ser otimizada. Lembre-se que o objetivo principal (*dashboard*) deve ser atendido.